In [1]:
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus

# leader_port = "/dev/tty.usbmodem575E0031751"
follower_port = "/dev/tty.usbserial-A10PCYXL"

# leader_arm = DynamixelMotorsBus(
#     port=leader_port,
#     motors={
#         # name: (index, model)
#         "shoulder_pan": (1, "xl330-m077"),
#         "shoulder_lift": (2, "xl330-m077"),
#         "elbow_flex": (3, "xl330-m077"),
#         "wrist_flex": (4, "xl330-m077"),
#         "wrist_roll": (5, "xl330-m077"),
#         "gripper": (6, "xl330-m077"),
#     },
# )

follower_arm = DynamixelMotorsBus(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)


In [2]:
follower_arm.connect()

Read failed due to communication error on port /dev/tty.usbserial-A10PCYXL for group_key ID_shoulder_pan_shoulder_lift_elbow_flex_wrist_flex_wrist_roll_gripper: [TxRxResult] There is no status packet!

Scanning all baudrates and motor indices
Setting bus baud rate to 9600. Previously 1000000.


100%|██████████| 252/252 [00:11<00:00, 21.30it/s]


Setting bus baud rate to 57600. Previously 9600.


100%|██████████| 252/252 [00:09<00:00, 27.44it/s]


Setting bus baud rate to 1000000. Previously 57600.


...

Detected motor with index 4
Setting its index to a temporary untaken index (11)


Scanning possible baudrates and motor indices


100%|██████████| 6/6 [00:00<00:00, 34.01it/s]


Detected motor with index 6
Setting its index to a temporary untaken index (12)


Scanning possible baudrates and motor indices


100%|██████████| 6/6 [00:00<00:00, 31.40it/s]


Detected motor with index 7
Setting its index to a temporary untaken index (13)

Setting expected motor indices: [1, 2, 3, 4, 5, 6]

Configuration is done!


In [3]:
follower_pos = follower_arm.read("Present_Position")
print(follower_pos)

[2139 2252 1936 2264  134 2701]


In [4]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

follower_arm.write("Torque_Enable", TorqueMode.ENABLED.value)

In [16]:
# Get the current position
position = follower_arm.read("Present_Position")

# Update first motor (shoulder_pan) position by +10 steps
position[0] += 10
follower_arm.write("Goal_Position", position)




In [26]:
# Update all motors position by -30 steps
position -= 30
follower_arm.write("Goal_Position", position)

# Update gripper by +30 steps
position[-1] += 30
follower_arm.write("Goal_Position", position[-1], "gripper")

In [27]:
follower_arm.write("Torque_Enable", TorqueMode.DISABLED.value)

In [28]:
follower_arm.disconnect()

In [2]:
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

robot = ManipulatorRobot(
    robot_type="koch",
    # leader_arms={"main": leader_arm},
    follower_arms={"main": follower_arm},
    calibration_dir=".cache/calibration/koch",
)

In [3]:
robot.connect()

Connecting main follower arm.
Missing calibration file '.cache/calibration/koch/main_follower.json'

Running calibration of koch main follower...

Move arm to zero position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_zero.webp

Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rotated.webp

Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/koch/follower_rest.webp

Calibration is done! Saving calibration file '.cache/calibration/koch/main_follower.json'
Activating torque on main follower arm.


In [4]:
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera

camera = OpenCVCamera(camera_index=1)
camera.connect()
color_image = camera.read()

print(color_image.shape)
print(color_image.dtype)

2024-09-10 20:17:18.165 python[73509:8350236] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


(720, 1280, 3)
uint8


In [4]:
robot.disconnect()

RobotDeviceNotConnectedError: ManipulatorRobot is not connected. You need to run `robot.connect()` before disconnecting.

In [2]:
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera

robot = ManipulatorRobot(
    follower_arms={"main": follower_arm},
    calibration_dir=".cache/calibration/koch",
    cameras={
        "laptop": OpenCVCamera(1, fps=30, width=640, height=480),
        "phone": OpenCVCamera(0, fps=30, width=640, height=480),
    },
)
robot.connect()

Connecting main follower arm.
Read failed due to communication error on port /dev/tty.usbserial-A10PCYXL for group_key ID_shoulder_pan_shoulder_lift_elbow_flex_wrist_flex_wrist_roll_gripper: [TxRxResult] There is no status packet!

Scanning all baudrates and motor indices
Setting bus baud rate to 9600. Previously 1000000.


100%|██████████| 252/252 [00:11<00:00, 21.39it/s]


Setting bus baud rate to 57600. Previously 9600.


100%|██████████| 252/252 [00:09<00:00, 27.52it/s]


Setting bus baud rate to 1000000. Previously 57600.


...

Detected motor with index 10
Setting its index to a temporary untaken index (15)




Scanning possible baudrates and motor indices


100%|██████████| 6/6 [00:00<00:00, 32.55it/s]


Detected motor with index 6
Setting its index to a temporary untaken index (16)

Setting expected motor indices: [1, 2, 3, 4, 5, 6]

Configuration is done!
Missing calibration file '.cache/calibration/koch/main_follower.json'

Running calibration of None main follower...

Move arm to zero position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/None/follower_zero.webp

Move arm to rotated target position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/None/follower_rotated.webp

Move arm to rest position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/None/follower_rest.webp

Calibration is done! Saving calibration file '.cache/calibration/koch/main_follower.json'
Activating torque on main follower arm.


/Users/wangxiaoyi/Documents/Coding/lerobot/lerobot/common/robot_devices/robots/manipulator.py:385: UserWarning: No preset found for robot type: None
  warnings.warn(f"No preset found for robot type: {self.robot_type}", stacklevel=1)
2024-09-10 20:39:53.219 python[86670:8397386] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [4]:
observation = robot.capture_observation()
print(observation["observation.images.laptop"].shape)
print(observation["observation.images.phone"].shape)
print(observation["observation.images.laptop"].min().item())
print(observation["observation.images.laptop"].max().item())

torch.Size([480, 640, 3])
torch.Size([480, 640, 3])
0
255


In [5]:
robot.disconnect()